In [1]:
# PREDIKCIA HODNOT FYZIKALNYCH PARAMETROV DOTYKOVYCH SYSTEMOCH METODOU k-NN S PRIDANYM SUMOM A BEZ SUMU
# VYTVORENIE A ULOZENIE MODELOV

In [1]:
# BLOK 1
# Importovanie kniznic.

import numpy as np
import pandas as pd

import pickle
import os

from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [2]:
# BLOK 2
# Funkcie pre generovanie sumu. Nastavenie pseudo-nahodneho generatora.

def generate_observation_sigma(space_obs_frac=0.5):
    """
    Draws a standard deviation of noise in light curve points from a "true" value provided in synthetic light curve.
    Noise sigma is drawn from bimodal distribution taking into account contributions from space based and earth based
    observations which have different levels of stochastic noise.

    :param space_obs_frac: ratio between earth based and space based observations
    :return: float; standard deviation of the light curve noise
    """
    earth_based_sigma = 4e-3
    space_based_sigma = 2e-4
    sigma = np.random.choice([earth_based_sigma, space_based_sigma], p=[1-space_obs_frac, space_obs_frac])
    return np.random.rayleigh(sigma)

def stochastic_noise_generator(curve):
    """
    Introduces gaussian noise into synthetic observation provided in `curve`.

    :param curve: numpy.array; normalized light curve
    :return: Tuple(numpy.array, float); normalized light curve with added noise, standard deviation of observations
    """
    sigma = generate_observation_sigma()
    return np.random.normal(curve, sigma), np.full(curve.shape, sigma)

np.random.seed(1234)

In [3]:
# BLOK 2
# Nacitanie vzorky dotykovych systemov

data_sample = pd.read_pickle("overcontact_curves_samples_knn.pkl").reset_index() 
len(data_sample)

500000

In [4]:
# BLOK 3
# Vyber a priprava dat. Skontrolovanie vyberu podla filtrov.

np.random.seed(1)

data_sample=data_sample.sample(n=100, random_state=1234)
data_sample["t2/t1"]=data_sample["secondary__t_eff"]/data_sample["primary__t_eff"]
data_sample=data_sample.round({"mass_ratio":14})


y = data_sample[["t2/t1", "inclination", "mass_ratio", "primary__surface_potential", "secondary__surface_potential"]]
X=[]
for row in data_sample["curve"]:
    X.append(row)

print(data_sample["filter"].value_counts())

Bessell_R    14
GaiaDR2      11
Kepler       10
SLOAN_r       9
TESS          8
Bessell_U     7
SLOAN_i       7
SLOAN_z       7
Bessell_I     6
Bessell_V     6
SLOAN_u       6
Bessell_B     5
SLOAN_g       4
Name: filter, dtype: int64


In [5]:
# BLOK 4 - a
# rozdelenie dat na trenovaciu a testovaciu mnozinu v pomere 80:20
# mnoziny pozostavaju z kriviek bez pridaneho sumu

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2
)

In [8]:
# BLOK 4 - b
# rozdelenie dat na trenovaciu a testovaciu mnozinu v pomere 80:20
# svetelnym krivkam je pridany umely sum

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X, y, test_size=0.2
)

X_train=[]
y_train=[]
for i in range(len(X_train_n)):
    for j in range(3):
        curve=stochastic_noise_generator(X_train_n[i])
        X_train.append(curve[0])
        y_train.append(y_train_n.iloc[i])
        j+=1        
X_train=np.array(X_train)
y_train=np.array(y_train)

X_test=[]
y_test=[]
for i in range(len(X_test_n)):
    for j in range(3):
        curve=stochastic_noise_generator(X_test_n[i])
        X_test.append(curve[0])
        y_test.append(y_test_n.iloc[i])
        j+=1
X_test=np.array(X_test)
y_test=np.array(y_test)

In [6]:
# Uprava testovacej mnoziny na dataframe

y_test = pd.DataFrame(y_test, columns=["t2/t1", "inclination", "mass_ratio", "primary__surface_potential", "secondary__surface_potential"])
y_test = y_test.reset_index()
y_test.head()

,index,t2/t1,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential
0,279060,0.904762,1.159527,1.250000,3.621534,3.621534
1,127849,0.965517,0.958100,2.500000,5.464111,5.464111
2,286650,0.931034,1.470485,1.111111,3.544514,3.544514
3,17315,0.958333,0.935198,0.700000,2.859116,2.859116
4,201970,1.000000,1.501347,2.000000,4.830454,4.830454


In [20]:
# BLOK 5
# Najdenie optimalnych hodnot hyper-parametrov pre model, vypis najlepsieho modelu

pipe_knn = Pipeline([('reg', MultiOutputRegressor(KNeighborsRegressor()))])
grid_param_knn = {
    "reg__estimator__n_neighbors": range(2, 7),
    "reg__estimator__metric": ["cosine", "euclidean", "l1"]
}
gs_svr = (GridSearchCV(estimator=pipe_knn, 
                      param_grid=grid_param_knn, 
                      ))
gs_svr = gs_svr.fit(X_train,y_train)
best_params = gs_svr.best_params_
gs_svr.best_estimator_  

Pipeline(steps=[('reg',
                 MultiOutputRegressor(estimator=KNeighborsRegressor(metric='cosine',
                                                                    n_neighbors=4)))])

In [8]:
best_params = {'reg__estimator__metric': 'cosine', 'reg__estimator__n_neighbors': 4}

In [9]:
# BLOK 6
# Vytvorenie architektury modelu. Spustenie trenovania na trenovacej mnozine. Vypis architektury modelu

knn_model = KNeighborsRegressor(n_neighbors=best_params['reg__estimator__n_neighbors'],
                                metric=best_params['reg__estimator__metric'], 
                                weights='distance')
knn_model.fit(X_train, y_train)

knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'cosine',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 4,
 'p': 2,
 'weights': 'distance'}

In [10]:
# BLOK 7
# Predikcia a vytvorenie dataframu z predikovanych hodnot

y_pred = knn_model.predict(X_test)

df_y_pred = pd.DataFrame(y_pred, columns=["T2/T1", "Inc", "mass_ratio", "PSP", "SSP"])
df_y_pred.head()

,T2/T1,Inc,mass_ratio,PSP,SSP
0,0.937556,1.230430,1.000000,3.235018,3.235018
1,0.976498,0.862225,1.191624,3.588923,3.588923
2,0.927420,1.460823,1.040322,3.351124,3.351124
3,0.927828,1.007173,1.076414,3.584951,3.584951
4,0.975379,1.448544,1.135275,3.602047,3.602047


In [11]:
# BLOK 8
# Chybovost pri predikcii pre kazdy parameter zvlast na testovacich krivkach

mse_t = mean_squared_error(y_test["t2/t1"], df_y_pred['T2/T1'])
mape_t = mean_absolute_percentage_error(y_test["t2/t1"], df_y_pred['T2/T1'])
mae_t = mean_absolute_error(y_test["t2/t1"], df_y_pred['T2/T1'])

mse_inc = mean_squared_error(y_test["inclination"], df_y_pred['Inc'])
mape_inc = mean_absolute_percentage_error(y_test["inclination"], df_y_pred['Inc'])
mae_inc = mean_absolute_error(y_test["inclination"], df_y_pred['Inc'])

mse_mass = mean_squared_error(y_test["mass_ratio"], df_y_pred['mass_ratio'])
mape_mass = mean_absolute_percentage_error(y_test["mass_ratio"], df_y_pred['mass_ratio'])
mae_mass = mean_absolute_error(y_test["mass_ratio"], df_y_pred['mass_ratio'])

mse_psp = mean_squared_error(y_test["primary__surface_potential"], df_y_pred['PSP'])
mape_psp = mean_absolute_percentage_error(y_test["primary__surface_potential"], df_y_pred['PSP'])
mae_psp = mean_absolute_error(y_test["primary__surface_potential"], df_y_pred['PSP'])

mse_ssp = mean_squared_error(y_test["secondary__surface_potential"], df_y_pred['SSP'])
mape_ssp = mean_absolute_percentage_error(y_test["secondary__surface_potential"], df_y_pred['SSP'])
mae_ssp = mean_absolute_error(y_test["secondary__surface_potential"], df_y_pred['SSP'])

print("MSE T2/T1: " + str(mse_t) + " MAE T2/T1: " + str(mae_t) + " MAPE T2/T1: " + str(mape_t * 100) + "%")
print("MSE INC: " + str(mse_inc) + " MAE INC: " + str(mae_inc) + " MAPE INC: " + str(mape_inc * 100) + "%")
print("MSE MASS: " + str(mse_mass) + " MAE MASS: " + str(mae_mass) + " MAPE MASS: " + str(mape_mass * 100) + "%")
print("MSE PSP: " + str(mse_psp) + " MAE PSP: " + str(mae_psp) + " MAPE PSP: " + str(mape_psp * 100) + "%")
print("MSE SSP: " + str(mse_ssp) + " MAE SSP: " + str(mae_ssp) + " MAPE SSP: " + str(mape_ssp * 100) + "%")

MSE T2/T1: 0.0009012387274669664 MAE T2/T1: 0.024293339139160486 MAPE T2/T1: 2.5750787257585688%
MSE INC: 0.005302649087321927 MAE INC: 0.06394439173710702 MAPE INC: 5.396354078245835%
MSE MASS: 1.257977497137111 MAE MASS: 0.7308877468252964 MAPE MASS: 118.9900143141897%
MSE PSP: 2.6196561220424237 MAE PSP: 1.0844692272465906 MAPE PSP: 32.408627388302556%
MSE SSP: 2.6196561220424237 MAE SSP: 1.0844692272465906 MAPE SSP: 32.408627388302556%


In [25]:
# BLOK 9
# ulozenie modelu

os.makedirs(os.path.dirname("models/knn_overcontact_noise_model.pkl"), exist_ok=True)
pickle.dump(knn_model, open("models/knn_overcontact_noise_model.pkl", 'wb'))